In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import os,sys,inspect

current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

import ipywidgets as widgets
from IPython.display import display

In [2]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
os.environ["ENV_FILE"] = '../../.env.dev'
from trade.db import TickerReturn, Ticker
from trade.db import DatabaseConnection
DatabaseConnection().connect()

True

In [4]:
import pendulum
start_period = pendulum.naive(2019, 10, 1)

In [5]:
tickers_list = ["ATD-B.TO", "MTL.TO", "T.TO", "DIS", "ASML", "TSM", "AAPL", "BRK-B", 
                "KMI", "BAM-A.TO", "BIP-UN.TO", "CNI", "CSU.TO", "X.TO", "MA", "MELI", "NVDA",
               "PYPL", "ASML", "MTY.TO"]
exlist = tickers_list

In [6]:
from trade.configuration import Configuration
exlist = Configuration().scrape().tickers

In [7]:
tickers = Ticker.select().where(Ticker.ticker.in_(exlist))
base_query = TickerReturn.select(
    TickerReturn.datetime,
    TickerReturn.close,
).where(
    TickerReturn.interval == '1d',
    TickerReturn.datetime > start_period
).order_by(
    TickerReturn.datetime.asc()
).join(Ticker)

In [9]:
dataframes = []
for ticker in exlist:
    ticker_query = base_query.where(TickerReturn.ticker == Ticker.get(Ticker.ticker == ticker))
    dataframe = pd.read_sql(ticker_query.sql()[0], DatabaseConnection(),
        params=ticker_query.sql()[1],
        index_col='datetime'
    )
    df = pd.DataFrame({ticker: dataframe.close}, index=dataframe.index)
    print(ticker)
    print(df.shape)
    dataframes.append(df)
dataframes = pd.concat(dataframes, axis=1) #.dropna()

T.TO
(61, 1)
BB
(60, 1)
ATD-B.TO
(61, 1)
REAL.TO
(61, 1)
QTRH.TO
(61, 1)
DRM.TO
(61, 1)
MX.TO
(61, 1)
KEY.TO
(61, 1)
FTT.TO
(61, 1)
BAM-A.TO
(61, 1)
BYD.TO
(61, 1)
L
(60, 1)
ADSK
(60, 1)
FSLY
(60, 1)
ANET
(60, 1)
ASML
(60, 1)
BERY
(60, 1)


In [10]:
ticker_returns = dataframes.pct_change().dropna()

In [11]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [12]:
mu = expected_returns.return_model(dataframes, method="capm_return")
S = risk_models.risk_matrix(dataframes, method="oracle_approximating")

In [13]:
ef = EfficientFrontier(mu, S)

In [14]:
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

In [15]:
cleaned_weights

OrderedDict([('T.TO', 0.04126),
             ('BB', 0.06549),
             ('ATD-B.TO', 0.0484),
             ('REAL.TO', 0.06305),
             ('QTRH.TO', 0.06043),
             ('DRM.TO', 0.04497),
             ('MX.TO', 0.06507),
             ('KEY.TO', 0.05389),
             ('FTT.TO', 0.06591),
             ('BAM-A.TO', 0.06145),
             ('BYD.TO', 0.05272),
             ('L', 0.05877),
             ('ADSK', 0.07071),
             ('FSLY', 0.06437),
             ('ANET', 0.05728),
             ('ASML', 0.06574),
             ('BERY', 0.06049)])

In [16]:
ef.portfolio_performance(verbose=True)

Expected annual return: 21.3%
Annual volatility: 20.1%
Sharpe Ratio: 0.96


(0.21336146720668753, 0.20059264012148137, 0.9639509559751817)

In [17]:
from pypfopt import DiscreteAllocation

da = DiscreteAllocation(cleaned_weights, dataframes.iloc[-1], total_portfolio_value=10000)
alloc, leftover = da.lp_portfolio()
print(f"Leftover: ${leftover:.2f}")
pd.Series(alloc).sort_index()

Leftover: $0.26


ADSK          2
ANET          2
ASML          1
ATD-B.TO     12
BAM-A.TO     11
BB           72
BERY         10
BYD.TO        2
DRM.TO       20
FSLY         10
FTT.TO       21
KEY.TO       21
L            11
MX.TO        14
QTRH.TO     250
REAL.TO      40
T.TO         16
dtype: int64